# Durham Arts and Culture Venues
## 1. Data Collection
## 2. Creating the Map

In [1]:

#!pip install geopandas
!pip install geopy
import requests
from geopy.geocoders import Nominatim

import types
import pandas as pd

import numpy as np
!pip install folium
import folium
import csv
from pandas import json_normalize #transform json file into a dataframe

print('imports done')


imports done


## 1. Data Collection
### - The Communities in Durham

In [2]:
municipalities = ['Ajax Ontario Canada',
                  'Brock Ontario Canada',
                  'Clarington Ontario Canada',
                  'Oshawa Ontario Canada',
                  'Pickering Ontario Canada',
                  'Scugog Ontario Canada',
                  'Uxbridge Ontario Canada',
                  'Whitby Ontario Canada']


#### A short function to retreive the latitude and longitude for 1 address

In [3]:

def get_lat_long(address):
    location = locator.geocode(address)
    if location != None:
        lat = location.latitude
        lon = location.longitude
    else:
        lat = np.nan
        lon = np.nan
    return lat, lon


In [4]:
# create a locator object
locator = Nominatim(user_agent='myGeocoder')

municipalities_locations = []
for m in municipalities:
    lat, lon = get_lat_long(m)
    municipalities_locations.append([m, lat, lon])
municipalities_locations

[['Ajax Ontario Canada', 43.850374, -79.024658],
 ['Brock Ontario Canada', 44.347685049999995, -79.10016490761303],
 ['Clarington Ontario Canada', 43.96868585, -78.65135738273909],
 ['Oshawa Ontario Canada', 43.91768, -78.862953],
 ['Pickering Ontario Canada', 43.835765, -79.090576],
 ['Scugog Ontario Canada', 44.1150554, -78.90598875892695],
 ['Uxbridge Ontario Canada', 44.1088502, -79.1226768],
 ['Whitby Ontario Canada', 43.899871, -78.940201]]

In [9]:
# build a database of venues for all the municipalities
# define foursquare arguments 
client_id ='client id'
client_secret = 'client secret'
version = '20180605'
limit = 100
radius = 10000

#create an empty dataframe
municipalities_venues = pd.DataFrame()

for lst in municipalities_locations:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        client_id,
        client_secret,
        version,
        lst[1],
        lst[2],
        radius,
        limit)
    results = requests.get(url).json()
    #extract the items
    venues = results['response']['groups'][0]['items']
    #create dataframe
    one_m_venues = json_normalize(venues)
    #add the new dataframe to the collector dataframe
    municipalities_venues = pd.concat([municipalities_venues,one_m_venues], axis=0)

municipalities_venues.shape 


(532, 24)

### - Extract the Relevant Columns

In [7]:
print(municipalities_venues.columns)

Index(['referralId', 'reasons.count', 'reasons.items', 'venue.id',
       'venue.name', 'venue.location.address', 'venue.location.lat',
       'venue.location.lng', 'venue.location.labeledLatLngs',
       'venue.location.distance', 'venue.location.postalCode',
       'venue.location.cc', 'venue.location.city', 'venue.location.state',
       'venue.location.country', 'venue.location.formattedAddress',
       'venue.categories', 'venue.photos.count', 'venue.photos.groups',
       'venue.venuePage.id', 'venue.location.crossStreet',
       'venue.events.count', 'venue.events.summary',
       'venue.location.neighborhood'],
      dtype='object')


In [10]:
filter_col=['venue.name','venue.location.address','venue.location.city','venue.categories','venue.location.lat','venue.location.lng']
municipalities_venues = municipalities_venues.loc[:,filter_col]
municipalities_venues.head()

,venue.name,venue.location.address,venue.location.city,venue.categories,venue.location.lat,venue.location.lng
0,Debbie's Boutique Cafe,94 Harwood Avenue South,Ajax,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.850183,-79.021350
1,Life Time Athletic,100 Beck Crescent,Ajax,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",43.859315,-79.013105
2,Portly Piper,235 Bayly St W,Ajax,"[{'id': '4bf58dd8d48988d157941735', 'name': 'N...",43.841323,-79.026583
3,Retro Burger,105 Bayly St W,Ajax,"[{'id': '4bf58dd8d48988d16c941735', 'name': 'B...",43.842269,-79.021905
4,Maimana Naan & Kabab,"279 Kingston Road East, Unit 7",Ajax,"[{'id': '503288ae91d4c4b30a586d67', 'name': 'A...",43.863670,-79.012364


In [ ]:
municipalities_venues.shape

In [11]:
#function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list=row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
#unpack the categories
municipalities_venues['venue.categories']=municipalities_venues.apply(get_category_type, axis=1)
municipalities_venues.head()

,venue.name,venue.location.address,venue.location.city,venue.categories,venue.location.lat,venue.location.lng
0,Debbie's Boutique Cafe,94 Harwood Avenue South,Ajax,Café,43.850183,-79.021350
1,Life Time Athletic,100 Beck Crescent,Ajax,Gym,43.859315,-79.013105
2,Portly Piper,235 Bayly St W,Ajax,New American Restaurant,43.841323,-79.026583
3,Retro Burger,105 Bayly St W,Ajax,Burger Joint,43.842269,-79.021905
4,Maimana Naan & Kabab,"279 Kingston Road East, Unit 7",Ajax,Afghan Restaurant,43.863670,-79.012364


In [13]:
#clean the columns
municipalities_venues.columns = [col.split('.')[-1] for col in municipalities_venues.columns]
municipalities_venues.head()

,name,address,city,categories,lat,lng
0,Debbie's Boutique Cafe,94 Harwood Avenue South,Ajax,Café,43.850183,-79.021350
1,Life Time Athletic,100 Beck Crescent,Ajax,Gym,43.859315,-79.013105
2,Portly Piper,235 Bayly St W,Ajax,New American Restaurant,43.841323,-79.026583
3,Retro Burger,105 Bayly St W,Ajax,Burger Joint,43.842269,-79.021905
4,Maimana Naan & Kabab,"279 Kingston Road East, Unit 7",Ajax,Afghan Restaurant,43.863670,-79.012364


In [14]:
#remove duplicate entries from overlap of radius
municipalities_venues.drop_duplicates(inplace=True)
municipalities_venues.shape

(354, 6)

In [18]:
#picked up some Toronto locations
indexNames = municipalities_venues[ municipalities_venues['city'] == 'Toronto' ].index
# Delete these row indexes from dataFrame
municipalities_venues.drop(indexNames , inplace=True)
municipalities_venues.shape

(327, 7)

In [16]:
municipalities_venues

,name,address,city,categories,lat,lng
0,Debbie's Boutique Cafe,94 Harwood Avenue South,Ajax,Café,43.850183,-79.021350
1,Life Time Athletic,100 Beck Crescent,Ajax,Gym,43.859315,-79.013105
2,Portly Piper,235 Bayly St W,Ajax,New American Restaurant,43.841323,-79.026583
3,Retro Burger,105 Bayly St W,Ajax,Burger Joint,43.842269,-79.021905
4,Maimana Naan & Kabab,"279 Kingston Road East, Unit 7",Ajax,Afghan Restaurant,43.863670,-79.012364
...,...,...,...,...,...,...
24,Zehrs,323 Toronto St S,Uxbridge,Grocery Store,44.089713,-79.130742
25,Walmart,6 Welwood Rd,Uxbridge,Department Store,44.087675,-79.133171
26,Uxbridge Arena,Brock St West,Uxbridge,Hockey Arena,44.107177,-79.133788
27,Mill Run Golf - Highland Course,NaN,Uxbridge,Golf Course,44.079908,-79.222929


In [29]:
municipalities_venues.reset_index(drop=True, inplace = True)
#municipalities_venues.drop('index',1, inplace = True)
municipalities_venues.head()

,name,address,city,categories,lat,lng
0,Debbie's Boutique Cafe,94 Harwood Avenue South,Ajax,Café,43.850183,-79.021350
1,Life Time Athletic,100 Beck Crescent,Ajax,Gym,43.859315,-79.013105
2,Portly Piper,235 Bayly St W,Ajax,New American Restaurant,43.841323,-79.026583
3,Retro Burger,105 Bayly St W,Ajax,Burger Joint,43.842269,-79.021905
4,Maimana Naan & Kabab,"279 Kingston Road East, Unit 7",Ajax,Afghan Restaurant,43.863670,-79.012364


In [21]:
cat = municipalities_venues['categories'].unique()
print(len(cat))
cat

98


array(['Café', 'Gym', 'New American Restaurant', 'Burger Joint',
       'Afghan Restaurant', 'Restaurant', 'Bookstore',
       'Mexican Restaurant', 'Breakfast Spot', 'Coffee Shop', 'Park',
       'Sports Bar', 'Garden Center', 'Gym / Fitness Center',
       'Grocery Store', 'Japanese Restaurant', 'Liquor Store',
       'Wings Joint', 'Warehouse Store', 'Brewery',
       'Middle Eastern Restaurant', 'Indian Restaurant', 'Ice Cream Shop',
       'Diner', 'Pizza Place', 'Fast Food Restaurant', 'Smoothie Shop',
       'Dessert Shop', 'American Restaurant', 'Pet Store',
       'Athletics & Sports', 'Movie Theater', 'Food',
       'Furniture / Home Store', 'Gastropub', 'Golf Course',
       'Chinese Restaurant', 'Bakery', 'Pharmacy', 'Seafood Restaurant',
       'BBQ Joint', 'Discount Store', 'Big Box Store', 'Pub',
       'Music Store', 'Bank', 'Soccer Stadium', 'Italian Restaurant',
       'Cosmetics Shop', 'Food Truck', 'Sandwich Place', 'Supermarket',
       'Lake', 'Gas Station', 'Reso

### Get the Categories Museum, Art Gallery, and Theater

In [30]:
museums = municipalities_venues.loc[municipalities_venues['categories']=='Museum']
museums

,name,address,city,categories,lat,lng
182,Oshawa Community Museum,1450 Simcoe St. South,Oshawa,Museum,43.863789,-78.827205


In [31]:
arts = municipalities_venues.loc[municipalities_venues['categories']=='Art Gallery']
arts

,name,address,city,categories,lat,lng
174,The Robert McLaughlin Gallery (@theRMG),72 Queen St,Oshawa,Art Gallery,43.895015,-78.865704
231,Station Gallery,1450 Henry Street,Whitby,Art Gallery,43.860443,-78.935325


In [32]:
theaters = municipalities_venues.loc[municipalities_venues['categories']=='Theater']
theaters

,name,address,city,categories,lat,lng
162,Oshawa Little Theatre,62 Russett Ave.,Oshawa,Theater,43.930277,-78.880503


In [33]:
arts_df = pd.concat([museums, arts, theaters], axis=0)
arts_df


,name,address,city,categories,lat,lng
182,Oshawa Community Museum,1450 Simcoe St. South,Oshawa,Museum,43.863789,-78.827205
174,The Robert McLaughlin Gallery (@theRMG),72 Queen St,Oshawa,Art Gallery,43.895015,-78.865704
231,Station Gallery,1450 Henry Street,Whitby,Art Gallery,43.860443,-78.935325
162,Oshawa Little Theatre,62 Russett Ave.,Oshawa,Theater,43.930277,-78.880503


## More Venues Needed
#### With only four venues from Foursquare a web search for more was done.  This resulted in a total of 32 venues which were stored in a CSV file.

In [48]:
# reading in the csv file

import csv

arts_df = pd.read_csv('Durham Data.csv')
arts_df.head()



,name,address,city,categories,lat,lng,Phone
0,Blue Willow Studio,19 Patricia Avenue,Oshawa,Art Gallery,43.901553,-78.849534,905 435-0575
1,Gallery 282 Ltd,321 Northcliffe Ave,Oshawa,Art Gallery,NaN,NaN,905 576-2187
2,Images 21 Art Studio,552 Westney Rd. N,Ajax,Art Gallery,NaN,NaN,(905) 239-6363
3,Magic Door Art Gallery,100 River Street,Sunderland,Art Gallery,NaN,NaN,(705) 357-1973
4,Marc Barrie Gallery,1613 7 Hwy,Brougham,Art Gallery,NaN,NaN,(905) 683-7010


In [49]:
#change the data types of the lat and lng columnss to float
arts_df[['lat','lng']] = arts_df[['lat','lng']].apply(pd.to_numeric)
arts_df.dtypes

name           object
address        object
city           object
categories     object
lat           float64
lng           float64
Phone          object
dtype: object

In [50]:
# find the latitude and longitude for each row in the arts_df

adds = arts_df['address'].to_list()
cities = arts_df['city'].to_list()
t_adds = [m+' '+n for m,n in zip(adds, cities)]
lat_lst =[]
long_lst = []
for ad in t_adds:
    #print(ad)
    lat, lng = get_lat_long(ad)
    #print(lat, lng)
    lat_lst.append(lat)
    long_lst.append(lng)


arts_df['lat']= lat_lst
arts_df['lng']= long_lst
arts_df.head()

,name,address,city,categories,lat,lng,Phone
0,Blue Willow Studio,19 Patricia Avenue,Oshawa,Art Gallery,43.901553,-78.849534,905 435-0575
1,Gallery 282 Ltd,321 Northcliffe Ave,Oshawa,Art Gallery,43.917164,-78.884608,905 576-2187
2,Images 21 Art Studio,552 Westney Rd. N,Ajax,Art Gallery,43.868353,-79.039990,(905) 239-6363
3,Magic Door Art Gallery,100 River Street,Sunderland,Art Gallery,54.875990,-1.421068,(705) 357-1973
4,Marc Barrie Gallery,1613 7 Hwy,Brougham,Art Gallery,54.648120,-2.729612,(905) 683-7010


In [51]:
#remove any NaN entries
arts_df.dropna(subset = ['lat','lng'], inplace = True)
arts_df

,name,address,city,categories,lat,lng,Phone
0,Blue Willow Studio,19 Patricia Avenue,Oshawa,Art Gallery,43.901553,-78.849534,905 435-0575
1,Gallery 282 Ltd,321 Northcliffe Ave,Oshawa,Art Gallery,43.917164,-78.884608,905 576-2187
2,Images 21 Art Studio,552 Westney Rd. N,Ajax,Art Gallery,43.868353,-79.039990,(905) 239-6363
3,Magic Door Art Gallery,100 River Street,Sunderland,Art Gallery,54.875990,-1.421068,(705) 357-1973
4,Marc Barrie Gallery,1613 7 Hwy,Brougham,Art Gallery,54.648120,-2.729612,(905) 683-7010
6,Preston Gallery,77 Brock St West,Uxbridge,Art Gallery,44.108679,-79.123945,905-550-8825
7,Ronden Art Gallery,133 Taunton Road West,Oshawa,Art Gallery,43.931302,-78.884629,905 718 9099
8,The LivingRoom Community Art Studio,149 Simcoe Street South,Oshawa,Art Gallery,43.893687,-78.861618,none
10,The Robert McLaughlin Gallery (RMG),72 Queen Street,Oshawa,Art Gallery,43.895721,-78.866684,(905) 576-3000
11,Visual Arts Centre of Clarington,143 Simpson Avenue,Bowmanville,Art Gallery,43.903348,-78.671832,(905) 623-5831


In [52]:
# save the dataframe to a csv file so it can be used in future codes

arts_df.to_csv('arts_venues_data.csv')
print('done')

done



# 2. Creating the Map

In [41]:
#find a central point for Durham
locator=Nominatim(user_agent='dh_explorer')
Durham_location = locator.geocode('Durham,Ontario,Canada')
Durham_location

Location(Durham, Oshawa, Durham Region, Golden Horseshoe, Ontario, Canada, (44.0338942, -78.86978798748362, 0.0))

In [42]:
D_lat = Durham_location.latitude
D_lon= Durham_location.longitude
print(D_lat, D_lon)

44.0338942 -78.86978798748362


In [43]:
# create map of Durham 
map_durham = folium.Map(location=[D_lat, D_lon], 
                        zoom_start =9,
                       )

In [44]:
# add legend from picture file
from folium.plugins import FloatImage
image_file = 'legend.PNG'

FloatImage(image_file, bottom=75, left=80).add_to(map_durham)


In [53]:
#add a title
loc = 'Durham Art and Culture Venues'
title_html = '''
             <h3 align="center" style="font-size:18px"><b>{}</b></h3>
             '''.format(loc)   

map_durham.get_root().html.add_child(folium.Element(title_html))


In [54]:
# add markers
for lat, lng, name, address, cat, city, phone in zip(arts_df['lat'], 
                                                     arts_df['lng'],
                                                     arts_df['name'],
                                                     arts_df['address'],
                                                     arts_df['categories'],
                                                     arts_df['city'],
                                                     arts_df['Phone']):
    
    #create the text for the popup labels
    if phone == 'none': #there is no phone number
        lab='''{},<br>
        {}, {}'''.format(name, address, city)
        
    else: #there is a phone number
        lab='''{},<br>
        {}, {}<br>
        {}'''.format(name, address, city, phone)
    
    iframe = folium.IFrame(lab,
                       width=150,
                       height=150)
    
    popup = folium.Popup(iframe,
                     max_width=200)
    
    #select the colours for the icons
    if cat == 'Art Gallery':
        colour = 'blue'
    elif cat == 'Museum':
        colour = 'red'
    else: #only Theater left
        colour = 'green'
   
    #create and add each marker
    folium.Marker(
        [lat, lng],
        radius = 5,
        popup = popup,
        icon=folium.Icon(color=colour,icon = 'fa-asterisk', prefix ='fa'),
        parse_html = False).add_to(map_durham)

map_durham

### Above map may not show the legend in the npviewer

### Convert map to HTML file

In [55]:
map_durham.save(outfile = 'Durham map.html')
print('done')

done
